# World Happiness App

### Project by: Adva Asraf, Dean Pianica, Maayan Raviv, Olga Kriyokov 
***

## Which country suites best for you?
The questionnaire below will ask you a few questions in regards to the importance of different variables in life in your own matter.

to add more content. 

### Invision link to our prototype: https://invis.io/WETEMASBK3M

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [2]:
df1=pd.read_csv('https://raw.githubusercontent.com/maayan3007/Capstone-DAV/master/world%20happiness%20report.csv')

In [3]:
df1.describe()

,Ladder,SD of Ladder,Positive affect,Negative affect,Social support,Freedom,Corruption,Generosity,Log of GDP per capita,Healthy life expectancy
count,156.000000,156.000000,155.000000,155.000000,155.000000,155.000000,148.000000,155.000000,152.000000,150.000000
mean,78.500000,78.500000,78.000000,78.000000,78.000000,78.000000,74.500000,78.000000,76.500000,75.500000
std,45.177428,45.177428,44.888751,44.888751,44.888751,44.888751,42.868014,44.888751,44.022721,43.445368
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,39.750000,39.750000,39.500000,39.500000,39.500000,39.500000,37.750000,39.500000,38.750000,38.250000
50%,78.500000,78.500000,78.000000,78.000000,78.000000,78.000000,74.500000,78.000000,76.500000,75.500000
75%,117.250000,117.250000,116.500000,116.500000,116.500000,116.500000,111.250000,116.500000,114.250000,112.750000
max,156.000000,156.000000,155.000000,155.000000,155.000000,155.000000,148.000000,155.000000,152.000000,150.000000


In [ ]:
def get_user_input_for_column(column, max):
  rating = 0
  
  while 1 > rating or max < rating:
    try:
        rating = int(input("{} 1-{}? : ".format(column,max)))
    except ValueError:
        print ("That wasn't an integer:(")
  return rating

### Since we're going to use KNN, we need the user's input to be on the same scale as the variables(which means number of ratings for that column)

Of course, for a user to enter numbers in such a high range(~155) is not ideal, since it's hard to choose the one correct number(which would be easier,  for example, in the 1-10 range), but we, as human beings, can tell if, for exapmle, we trust the government not to be corrupt with a value/score of 83 or more like 85; we do have that feeling

In [ ]:
print("Please answer all the questions. REMEMBER, \n1 means that this feature contributes in a maximum level to your happiness!\n\n")
# Social Support - The extent to which Social support contributed to the calculation of the Happiness Score.
social_support_answer = get_user_input_for_column('Social Support: How much does the government contribute to the quality of family life, nuclear and joint family on a scale of', 155) / 155

# Freedom - The extent to which Freedom contributed to the calculation of the Happiness Score.
Freedom_answer = get_user_input_for_column('Freedom: In your future country, how much will an individual be able to conduct himself based on his free will? On a scale of', 155) / 155

# Corruption - The extent to which Perception of Corruption contributes to Happiness Score.
Corruption_answer = get_user_input_for_column('Corruption: How much do you trust your government to be corruption free? On a scale of', 148) / 148

# Generosity The extent to which Generosity contributed to the calculation of the Happiness Score.
Generosity_answer = get_user_input_for_column('Generosity: In your future country, how much involved in peacekeeping and global aid do you want your country to be? On a scale of', 155) / 155

# Log of GDP per capita - The extent to which GDP contributes to the calculation of the Happiness Score.
GDP_per_capita_answer = get_user_input_for_column('GDP per capita: In your future country, how important is quality of life based on the GDP? On a scale of', 152) / 152

# Healthy life expectancy - The extent to which Life expectancy contributed to the calculation of the Happiness Score.
Healthy_life_expectancy_answer = get_user_input_for_column('Healthy life expectancy: How important is the healthcare and average life expectancy in your future country? on a scale of', 150) / 150

Please answer all the questions. REMEMBER, 
1 means that this feature contributes in a maximum level to your happiness!




In [ ]:
user_input_vector = np.array([social_support_answer, Freedom_answer, Corruption_answer, Generosity_answer, 
                               GDP_per_capita_answer, Healthy_life_expectancy_answer])

In [ ]:
questionnaire_df = df1.iloc[:,5:11]

In [ ]:
questionnaire_df['Country (region)'] = df1['Country (region)']

In [ ]:
#  Because we only care about full rows which provide us all the information so we could use it with the KNN algorithm.
questionnaire_df = questionnaire_df.dropna()

In [ ]:
# standart normalization function to define all values to the same range 0-1
def normalize(df):
  new_df = []
  min = 1
  for i in df:
    new_df.append((i - min) / (df.max() - min))
  return new_df  

In [ ]:
questionnaire_df['norm_social_support'] = normalize(questionnaire_df['Social support'])
questionnaire_df['norm_Freedom'] = normalize(questionnaire_df['Freedom'])
questionnaire_df['norm_Corruption'] = normalize(questionnaire_df['Corruption'])
questionnaire_df['norm_Generosity'] = normalize(questionnaire_df['Generosity'])
questionnaire_df['norm_GDP_per_capita'] = normalize(questionnaire_df['Log of GDP\nper capita'])
questionnaire_df['norm_Healthy_life_expectancy'] = normalize(questionnaire_df['Healthy life\nexpectancy'])

In [ ]:
norm_questionnaire_df = questionnaire_df.iloc[:,6:]

In [ ]:
# we build this dictionary to map between the output prediction value(number) and the country associated with it
number_to_country_dictionary = {}
i = 0
for country in norm_questionnaire_df['Country (region)']:
  number_to_country_dictionary[i] = country
  i = i + 1

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# we will find the k nearest neighbors, which are the k country vectors(meaning the the coutry's 6 variables values) that have the smallest distance from our point.
model = KNeighborsClassifier(n_neighbors=i)

# Train the model using the training sets
model.fit(norm_questionnaire_df.iloc[:,1:],list(norm_questionnaire_df['Country (region)']))
#Predict Output
user_input_vector = user_input_vector.reshape(1, -1)
k_matching_countries = model.kneighbors(user_input_vector,n_neighbors=3)
print("1. ", number_to_country_dictionary[k_matching_countries[1][0][0]])
print("2. ", number_to_country_dictionary[k_matching_countries[1][0][1]])
print("3. ", number_to_country_dictionary[k_matching_countries[1][0][2]])

